In [1]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
import time 
import os
import json
import base64

#from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def get_pdf_links_one_page(url, element_class, driver = None, button_xpath = None, pdf_dict = {}, search_word = None):
    if driver is None:
        driver = webdriver.Chrome()
    
    if not driver.current_url == url:
        driver.get(url)
    
    # wait till elements show up
    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CLASS_NAME, element_class)))
    #time.sleep(10)
    
    #scoll down a little
    #driver.execute_script("window.scrollTo(0, 0.25*document.body.scrollHeight);")

    
    # Find all elements with a specific class name
    elements = driver.find_elements(By.CLASS_NAME, element_class)
    #print(elements)
    
    # Loop through each element
    for element in elements:
        # Click on the element
        pdf_name = element.find_element(By.TAG_NAME, 'span').text
        if not search_word is None and search_word not in pdf_name: #skip elements that don't contain keyword
            continue
        if len(pdf_name) < 3: #skip 热点信息
            continue
        print(pdf_name)
        try:
            element.find_element(By.TAG_NAME, 'span').click()
        except WebDriverException:
            print(pdf_name + " is not clickable")
            continue
        time.sleep(5)  # wait for page to load
        #WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, button_xpath)))
        if button_xpath is None:
            pdf_button = driver.find_elements(By.TAG_NAME, 'button')[-1]
        else:
            pdf_button = driver.find_element(By.XPATH, button_xpath)
        try:
            #WebDriverWait(driver, 10).until(EC.element_to_be_clickable(pdf_button))
            pdf_button.click()
        except WebDriverException:
            # no pdf link, save screenshot instead
            try:
                img = driver.find_elements(By.TAG_NAME, 'div')[-3].screenshot_as_base64
                pdf_link = base64.decodebytes(img.encode('utf-8'))
                print(pdf_name+' has screenshot saved')
            except WebDriverException:
                pdf_link = None
                print(pdf_nam + 'has no Link and cannot screenshot')
            pdf_dict[pdf_name] = pdf_link
            #close popup
            try:
                driver.find_elements(By.CSS_SELECTOR, "[aria-label=Close]")[1].find_element(By.TAG_NAME, 'i').click()
                time.sleep(3)
            except:
                print(pdf_name)
            continue
        time.sleep(5)
        #print(driver.current_url)

        
        driver.switch_to.window(driver.window_handles[-1])
        pdf_link = driver.current_url
        driver.close()
        try:
            driver.switch_to.window(driver.window_handles[0])
        except:
            ...
            
        #close popup
        time.sleep(5)
        driver.find_elements(By.CSS_SELECTOR, "[aria-label=Close]")[1].find_element(By.TAG_NAME, 'i').click()
        time.sleep(3)
        pdf_dict[pdf_name] = pdf_link
    return pdf_dict, driver

def get_pdf_links_more_pages(url, element_class, driver = None, page_num = 1, pdf_dict = {}, search_word = None):
    for _ in range(page_num-1):
        pdf_dict, driver = get_pdf_links_one_page(url, element_class, driver = driver, pdf_dict = pdf_dict, search_word=search_word)
        if driver.find_element(By.CLASS_NAME, 'btn-next').is_enabled():
            driver.find_element(By.CLASS_NAME, 'btn-next').click()
            time.sleep(5)
    return pdf_dict

def save_pdf(pdf_dict, folder_path = None):
    if folder_path is None:
        folder_path = os.curdir
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    for k, v in pdf_dict.items():
        if v is None:
            continue
        if type(v) == str:
            path = os.path.join(folder_path, k+'.pdf')
            pdf = requests.get(v, verify = False).content
            with open(path, 'wb') as file:
                file.write(pdf)
                print('save at '+path)
        elif type(v) == bytes:
            path = os.path.join(folder_path, k+'.png')
            with open(path, 'wb') as file:
                file.write(v)
                #file.write(base64.decodebytes(v))
                print('save at '+path)


In [3]:
pmos_url = "https://pmos.sd.sgcc.com.cn/pxf-settlement-outnetpub/#/pxf-settlement-outnetpub/columnHomeLeftMenuNew"
element_class = 'el-table__row'
button_xpath = '/html/body/div[3]/div/div[2]/div/div/div[2]/div[3]/span[2]/button[2]'
search_word = '工作日报'

In [4]:
%%time
pdfs = get_pdf_links_more_pages(pmos_url, element_class, page_num=2, search_word=None)

山东电力现货市场2023年4月结算试运行工作日报(4月19日)
2023年3月份绿电绿证交易信息披露
山东电力现货市场2023年4月结算试运行工作日报(4月18日)
山东电力现货市场2023年4月结算试运行工作日报(4月16日)
山东电力现货市场2023年4月结算试运行工作日报(4月17日)
山东电力现货市场2023年4月结算试运行工作日报(4月15日)
山东电力交易中心有限公司关于公示山东安悦节能技术有限公司等5家注册信息变更的公告
山东电力现货市场2023年4月结算试运行工作日报(4月14日)
山东电力交易中心有限公司关于发布山东电亮亮售电有限公司市场注销公示结果的公告
山东电力交易中心有限公司关于发布河北华瑞能源集团有限公司等2家售电公司减少售电业务范围公示结果的公告
Wall time: 3min 28s


In [58]:
len(pdfs)

40

In [ ]:
test_dict={'山东电力现货市场2023年4月结算试运行工作日报(4月11日)': 'https://pmos.sd.sgcc.com.cn/px-settlement-infpubmeex/fileService/preview?fileId=nf80340850b2c4d2ebdaf97144353b481#toolbar=0', '电网企业代理非市场用户每个交易时段的购、售电量': b'...'}

In [ ]:
for k, v in test_dict.items():
    if type(v) == str:
        pdf = requests.get(v, verify = False).content
        with open(k+'.pdf', 'wb') as file:
            file.write(pdf)
    if type(v) == bytes:
        img = base64.decodebytes(v)
        with open(k+'.png', 'wb') as file:
            file.write(img)

In [ ]:
b = test_dict['电网企业代理非市场用户每个交易时段的购、售电量']
img = base64.decodebytes(b)
with open('test'+'.png', 'wb') as file:
    file.write(img)

In [ ]:
save_pdf(test_dict)

In [ ]:
pdfs

In [10]:
save_pdf(pdfs)

C:\Users\Bill Chen\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pmos.sd.sgcc.com.cn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


save at .\山东电力现货市场2023年4月结算试运行工作日报(4月14日).pdf


C:\Users\Bill Chen\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pmos.sd.sgcc.com.cn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


save at .\山东电力现货市场2023年4月结算试运行工作日报(4月13日).pdf


C:\Users\Bill Chen\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pmos.sd.sgcc.com.cn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


save at .\山东电力现货市场2023年4月结算试运行工作日报(4月12日).pdf


C:\Users\Bill Chen\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pmos.sd.sgcc.com.cn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


save at .\山东电力现货市场2023年4月结算试运行工作日报(4月11日).pdf


In [ ]:
pdf411 = requests.get('https://pmos.sd.sgcc.com.cn/px-settlement-infpubmeex/fileService/preview?fileId=nf80340850b2c4d2ebdaf97144353b481#toolbar=0', verify = False).content

In [ ]:
with open('411.pdf', 'wb') as file:
    file.write(pdf411)

In [ ]:
cq_pmos = 'https://pmos.cq.sgcc.com.cn/pxf-settlement-outnetpub/#/pxf-settlement-outnetpub/columnHomeLeftMenuNew'
element_class = 'el-table__row'

In [ ]:
cq= get_pdf_links_more_pages(cq_pmos, element_class, page_num=5)

In [ ]:
cq

In [30]:
driver = webdriver.Chrome()

In [31]:
driver.get(pmos_url)

In [32]:
driver.find_element(By.CLASS_NAME, 'btn-next').click()

In [33]:
elements = driver.find_elements(By.CLASS_NAME, element_class)

In [35]:
for element in elements:
    pdf_name = element.find_element(By.TAG_NAME, 'span').text
    
    print(pdf_name)

山东电力交易中心有限公司关于公示第六十八批申请注册售电公司信息的公告
山东电力现货市场2023年4月结算试运行工作日报(4月13日)
2023年4月12日绿电绿证交易座谈会
山东电力现货市场2023年4月结算试运行工作日报(4月12日)
山东电力现货市场2023年4月结算试运行工作日报(4月11日)
2023年3月省间交易输电价格方案
2023年4月省间交易输电价格方案
绿色电力证书交易用户手册
山东电力现货市场2023年4月结算试运行工作日报(4月10日)
山东电力现货市场2023年4月结算试运行工作日报（4月9日）



4
5
6
7
8
9
10


In [46]:
elements[2].find_element(By.TAG_NAME, 'span').click()

In [37]:
driver.find_elements(By.CSS_SELECTOR, "[aria-label=Close]")

[<selenium.webdriver.remote.webelement.WebElement (session="f97b1e40806c911e352f86ed6f334624", element="5cc3a44b-08ec-47b1-8501-d6a2bfe28039")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f97b1e40806c911e352f86ed6f334624", element="d465c2cb-a289-4727-81f3-1bee0aff5299")>]

In [38]:
driver.execute_script("window.scrollTo(0, 0.25*document.body.scrollHeight);")

In [ ]:
driver.find_element(By.CLASS_NAME, 'btn-next').is_enabled()

In [ ]:
all_rows = driver.find_elements(By.CLASS_NAME,'el-table__row')

In [ ]:
all_rows[5].find_element(By.TAG_NAME,'span').text

In [ ]:
all_rows[5].click()

In [ ]:
img = driver.find_elements(By.TAG_NAME, 'div')[-3].screenshot_as_base64

In [ ]:
import base64

In [ ]:
img.encode('utf-8')

In [ ]:
type(img.encode('utf-8'))

In [ ]:
fh = open("imageToSave.png", "wb")
fh.write(base64.decodebytes(img.encode('utf-8')))
fh.close()

# or, more concisely using with statement
#with open("imageToSave.png", "wb") as fh:
#    fh.write(img.decode('base64'))

In [ ]:
driver.find_elements(By.TAG_NAME, 'div')[-3].screenshot

In [ ]:
driver.find_elements(By.TAG_NAME, 'div')[-3].screenshot_as_png

In [ ]:
button = driver.find_element(By.XPATH,'/html/body/div[3]/div/div[2]/div/div/div[2]/div[3]/span[2]/button[2]')

In [ ]:
button.click()

In [ ]:
driver.switch_to.window(driver.window_handles[-1])
print(driver.current_url)
driver.close()
driver.switch_to.window(driver.window_handles[0])

In [ ]:
popup_close

In [ ]:
/html/body/div[4]/div/div[1]/button/i

In [ ]:
driver.find_elements(By.CSS_SELECTOR, "[aria-label=Close]")[1].find_element(By.TAG_NAME, 'i').click()

In [ ]:
driver.find_element(By.XPATH, '/html/body/div[3]/div/div[1]/button/i').click()

In [ ]:
all_rows[1].click()

In [ ]:
driver.find_element(By.XPATH, '/html/body/div[4]/div/div[1]/button')

In [ ]:
driver.find_element(By.XPATH,'/html/body/div[4]/div/div[1]/button/i').click()

In [ ]:
close_buttons = driver.find_elements(By.CSS_SELECTOR, "[aria-label=Close]")

In [ ]:
close_buttons

In [ ]:
close_buttons

In [ ]:
close_buttons[1].click()

In [ ]:
[button.label for button in buttons]

In [ ]:
driver.switch_to.default_content()

In [ ]:
button[-1].click()

In [ ]:
pdf_url = driver.current_url

In [ ]:
os.path.basename(pdf_url)

In [ ]:
pdf = requests.get(pdf_url, verify = False).content
with open('pdf1.pdf', 'wb') as file:
    file.write(pdf)

In [ ]:
pdf_url = 'https://pmos.cq.sgcc.com.cn/px-settlement-infpubmeex/fileService/preview?fileId=nd720eea3e6cc4e9b8c171df0915aac1d#toolbar=0'

In [ ]:
driver.window_handles

In [ ]:
driver.switch_to.window(driver.window_handles[0])